題目: 將某篇文章以上下文相同，比方三連詞(trigram)方式修改內容<br>
說明: 某篇文章中我們可以找出所有的三連詞(trigram)，以及在前字與後字出現時，<br>
按照出現度隨機選出一個字去換掉中間字，這是利用三連詞修改文章內容的最基本作法。<br>
一旦字典的資料結構建立，我們就以某種機率(比方20%)去置換原文，並將置換文與原文印出來<br>
<br>
延伸: 可用五連詞或七連詞去取代中間字，可利用三連詞之前兩字去更換第三字，<br>
可增加詞性的相同性(Parts Of Speech)提高可讀性，甚至使用Word2Vec, Glove，或者RNN的??<br>
<br>
範例程式檔名: article_modifier_自動文件修改器.py<br>
模組: sklearn, random, numpy, nltk, bs4<br>
輸入檔：positive(作業數據).review<br>
成績: 被置換文的合理性與可讀性<br>

使用三連詞 trigrams 練習簡易文件產生器

In [1]:
import nltk
nltk.download('punkt')

import random
import numpy as np
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


load the reviews

In [2]:
positive_reviews = BeautifulSoup(open('positive(作業數據).review', encoding='ISO-8859-1').read(), "lxml")
positive_reviews = positive_reviews.findAll('review_text')

提出**三連詞**並置入字典<br>
(w1, w2)當作key,[w3]當作值

In [3]:
trigrams = {}
for review in positive_reviews:
    s = review.text.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    for i in range(len(tokens) - 2):
        k = (tokens[i], tokens[i + 1])
        if k not in trigrams:
            trigrams[k] = []
        trigrams[k].append(tokens[i + 2])

將最後字矩陣變成似然機率(likelihood)向量

In [4]:
trigram_probabilities = {}
for k, words in trigrams.items():
    # 產生一個 word -> count 字典
    if len(set(words)) > 1:
        # 如果最後字(last word)不只有一個機率 
        d = {}
        n = 0
        for w in words:
            if w not in d:
                d[w] = 0
            d[w] += 1
            n += 1
        for w, c in d.items():
            d[w] = float(c) / n
        trigram_probabilities[k] = d

In [5]:
def random_sample(d):
    # 從字典隨機選出一個帶機率值的樣本，回傳累積機率值最大的字
    r = random.random()
    cumulative = 0
    for w, p in d.items():
        cumulative += p
        if r < cumulative:
            return w

In [6]:
def test_spinner():
    review = random.choice(positive_reviews)
    s = review.text.lower()
    print('Original:', s)
    tokens = nltk.tokenize.word_tokenize(s)
    for i in range(len(tokens) - 2):
        if random.random() < 0.2:   # 20% chance of replacement
            k = (tokens[i], tokens[i + 1])
            if k in trigram_probabilities:
                w = random_sample(trigram_probabilities[k])
                tokens[i + 1] = w
    print('Spun:')
    print(' '.join(tokens).replace(' .', '.').replace(' \'', '\'').replace(' ,', ',').replace('$ ', '$').replace(' !', '!'))

In [7]:
test_spinner()

Original: 
i had no problem with the product.  the directions were pretty simple.  

problem:  i never received my rebate information!!!


Spun:
i had no problem with the description. the directions were pretty simple. problem : i never received my rebate information!!!
